## Demonstration of RL implementation for PRISM models

This demonstration should provide general idea of how to use this implementation for own experiments with RL training.

Remark: This demonstration does not include the important part of the project of using verification with PAYNT/Storm and focuses solely on training RL agents on PRISM models.

### Imports and Initial Setting

In [48]:
import tf_agents # The RL library
import tensorflow as tf

from rl_src.agents.recurrent_ppo_agent import Recurrent_PPO_agent # The recurrent PPO agent
from rl_src.tools.args_emulator import ArgsEmulator # The argument emulator (historically I supported command line arguments for all of them)
from rl_src.environment.environment_wrapper_vec import EnvironmentWrapperVec # The vectorized environment wrapper

from rl_src.tests.general_test_tools import *
from rl_src.tools.evaluators import evaluate_policy_in_model
from rl_src.interpreters.extracted_fsc.table_based_policy import TableBasedPolicy 

import os


First, we have to create parameters for training using ArgsEmulator. Note, that the used parameters are only the subset of all possible Arguments, but there are many more, which are perhaps more experimental. 

In [49]:
prism_path = "./rl_src/models/intercept" # Folder with the PRISM model, you can change this to your own path
prism_template = os.path.join(prism_path, "sketch.templ") # File with the prism template
prism_spec = os.path.join(prism_path, "sketch.props") # File with the task PTL specification

args = ArgsEmulator(
    prism_model=prism_template,
    prism_properties=prism_spec,
    constants="", # If PRISM model has some unresolved constants (such as Size=N, you can set them as "N=10")
    discount_factor=0.99,
    learning_rate=1.6e-4,
    trajectory_num_steps=32, # Number of steps in a single driver.run() call and the length of the trajectory (sub-episodes) for training the agent
    num_environments=16, # Number of environments to run in parallel/vectorized (batch size currently corresponds to this), I usually use 256
    batch_size=16,
    max_steps=400 # Number of steps, that can be taken in the environment per episode (if more, the episode is truncated)
)

Now we can initialize the model.

In [50]:
stormpy_model = initialize_prism_model(args.prism_model, args.prism_properties, args.constants) # Initialize the PRISM model using stormpy
# stormpy_model = any POMDP storm model, if you have some constructed model -- it is useful in PAYNT, since PAYNT constructs its own POMDPs and I just take them.
print(stormpy_model) # Prints the description of the PRISM model



INFO:environment.pomdp_builder:Construct POMDP representation...


-------------------------------------------------------------- 
Model type: 	POMDP (sparse)
States: 	4705
Transitions: 	18386
Choices: 	11810
Observations: 	2598
Reward Models:  steps
State Labels: 	5 labels
   * deadlock -> 0 item(s)
   * init -> 1 item(s)
   * notbad -> 4607 item(s)
   * goal -> 96 item(s)
   * exits -> 98 item(s)
Choice Labels: 	7 labels
   * placement -> 1 item(s)
   * adv -> 2256 item(s)
   * east -> 2352 item(s)
   * north -> 2401 item(s)
   * south -> 2352 item(s)
   * west -> 2352 item(s)
   * finished -> 96 item(s)
-------------------------------------------------------------- 



Now we can show some basic work with the environment. 

In [51]:
env = EnvironmentWrapperVec(stormpy_model, args, num_envs=args.num_environments)
tf_env = TFPyEnvironment(env)
print("Environment initialized")

# Example of how to use the environment
print(f"Initial TimeStep: {tf_env.reset()}")
print(f"TimeStep after some arbitrary step {tf_env.step(tf.zeros((args.num_environments,)))}") # The number of actions is equal to the number of environments


INFO:environment.vectorized_sim_initializer:Model intercept found in compiled_models_vec_storm. The model will be loaded
ERROR:environment.environment_wrapper_vec:Grid-like renderer not possible to initialize.


Simulator initialized.


INFO:environment.environment_wrapper_vec:Resetting the environment.


<class 'environment.environment_wrapper_vec.EnvironmentWrapperVec'>
Environment initialized
Initial TimeStep: TimeStep(
{'step_type': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)>,
 'reward': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>,
 'discount': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99,
       0.99, 0.99, 0.99, 0.99, 0.99], dtype=float32)>,
 'observation': {'observation': <tf.Tensor: shape=(16, 8), dtype=float32, numpy=
array([[ 0.,  0.,  3.,  0., -1., -1.,  0.,  0.],
       [ 0.,  0.,  3.,  0., -1., -1.,  0.,  0.],
       [ 0.,  0.,  3.,  0., -1., -1.,  0.,  0.],
       [ 0.,  0.,  3.,  0., -1., -1.,  0.,  0.],
       [ 0.,  0.,  3.,  0., -1., -1.,  0.,  0.],
       [ 0.,  0.,  3.,  0., -1., -1.,  0.,  0.],
       [ 0.,  0.,  3.,  0., -1., -1

Important thing in this framework is the structure of TimeSteps

In [52]:
print(tf_env.time_step_spec())

TimeStep(
{'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': {'observation': TensorSpec(shape=(8,), dtype=tf.float32, name='observation'),
                 'mask': TensorSpec(shape=(7,), dtype=tf.bool, name='mask'),
                 'integer': TensorSpec(shape=(1,), dtype=tf.int32, name='integer_information')}})


As you can see above, the TimeStep consists of four parts:
 * step_type: information about the state of the simulation (i.e. simulation just started as 0, simulation is in the middle of run as 1 and simulation ended in 2). However, it is important to note, that since I work with vectorized environment, if you play some arbitrary action when the simulation ended, the simulation restarts to initial state.
 * reward: information about the reward that the agent receives. Note, that this is the value that the agent is trained on, but in the training output, I usually provide also real reward (the reward of the model without some training-related reward stuff).
 * discount: discount :)
 * observation: dictionary of three important vectors:
   * observation: valuations of features of provided observation
   * integer: index of observation in StormPy POMDP model
   * mask: vector of booleans describing which actions are currently legal to play (if the action is illegal, simulator plays selects random legal action)

In the next cell, we initialize the Recurrent PPO Agent. Note, that I provide both env and tv_env (original wrapper and TensorFlow version of the environment). I do it, because my environment provides some other relevant information about the simulation, that is used during the evaluation. However, the training is runned on the tf_env.  

In [53]:
agent = Recurrent_PPO_agent(
            env, tf_env, args)
agent.action(tf_env.reset()) # Example of how to use the policy


INFO:root:Agent initialized
INFO:root:Replay buffer initialized
INFO:root:Collector driver initialized
INFO:root:Evaluation driver initialized
INFO:environment.environment_wrapper_vec:Resetting the environment.


PolicyStep(action=<tf.Tensor: shape=(16,), dtype=int32, numpy=array([3, 6, 1, 0, 1, 5, 6, 3, 1, 1, 1, 1, 3, 2, 3, 2], dtype=int32)>, state={'actor_network_state': [<tf.Tensor: shape=(16, 32), dtype=float32, numpy=
array([[ 6.57125175e-01,  2.45846752e-02, -5.04158258e-01,
         5.65134168e-01,  2.27224872e-01, -4.64851975e-01,
         6.54574394e-01, -3.13785486e-02, -6.00984460e-03,
         6.76215142e-02,  2.19584897e-01, -2.13127583e-01,
         1.34125501e-01,  1.27421722e-01,  6.32455170e-01,
        -5.68710901e-02, -3.32760900e-01,  1.14051085e-02,
         1.01076784e-02,  4.59009269e-03,  7.01762974e-01,
        -2.17506438e-01, -1.27916457e-02,  6.08660817e-01,
         6.48001730e-02, -1.83698714e-01, -4.05178756e-01,
        -4.63577598e-01,  1.43414155e-01, -2.41662916e-02,
         9.13439319e-03, -8.58846834e-05],
       [ 6.57125175e-01,  2.45846752e-02, -5.04158258e-01,
         5.65134168e-01,  2.27224872e-01, -4.64851975e-01,
         6.54574394e-01, -3.1378548

## Agent Usage

In [54]:
agent.train_agent(iterations = 100) # Train the agent for nr runs
# I recommend to use at least 1000 iterations in general, but it depends on the task and the model


INFO:agents.father_agent:Training agent with replay buffer option: 1
INFO:agents.father_agent:Before training evaluation.
INFO:environment.environment_wrapper_vec:Resetting the environment.
INFO:agents.father_agent:Average Return = -77.77941131591797
INFO:agents.father_agent:Average Virtual Goal Value = 36.02941131591797
INFO:agents.father_agent:Goal Reach Probability = 0.7205882352941176
INFO:agents.father_agent:Trap Reach Probability = 0.27941176470588236
INFO:agents.father_agent:Variance of Return = 2574.877685546875
INFO:agents.father_agent:Current Best Return = -77.77941131591797
INFO:agents.father_agent:Current Best Reach Probability = 0.7205882352941176
INFO:environment.environment_wrapper_vec:Resetting the environment.
INFO:agents.father_agent:Training agent on-policy
INFO:agents.father_agent:Step: 0, Training loss: 2.475553512573242
INFO:environment.environment_wrapper_vec:Resetting the environment.
INFO:agents.father_agent:Average Return = -80.984375
INFO:agents.father_agent:

## Basic inference loop, where you can collect whatever you want. 

In TF Agents, there are some other options how to run policy (primarily drivers) and how to store data (replay buffers), but for simplicity, I show only the simplest running loop possible.

Note, that I use agent.policy(), since there are multiple options how to work with the agent, because beside the original policy I also use wrapper masked policy, that can provide functionality with masking, reward machines etc.

In [55]:
time_step = tf_env.reset() # Reset the environment before evaluation
policy_state = agent.policy().get_initial_state(tf_env.batch_size) # Get the initial state of the policy


for i in range(10):
    policy_step = agent.policy().action(time_step, policy_state) # Get the action from the policy
    print(f"Policy step {i}: {policy_step.action}") # Print the action
    policy_state = policy_step.state # Update the policy state
    time_step = tf_env.step(policy_step.action) # Step the environment with the action

INFO:environment.environment_wrapper_vec:Resetting the environment.


Policy step 0: [4 2 1 1 5 6 6 1 4 6 1 3 0 1 3 1]
Policy step 1: [2 5 2 6 2 1 6 4 4 6 2 2 2 6 4 2]
Policy step 2: [3 2 3 6 5 5 4 5 1 1 5 1 2 0 5 0]
Policy step 3: [1 2 4 0 1 2 2 3 1 1 1 2 5 1 1 2]
Policy step 4: [1 5 1 5 0 1 5 0 4 5 4 1 5 5 0 0]
Policy step 5: [3 5 1 3 0 0 4 6 4 6 5 4 3 1 0 3]
Policy step 6: [1 1 3 2 1 5 1 2 1 2 1 1 5 4 0 3]
Policy step 7: [3 2 0 5 3 4 2 1 6 4 3 0 0 6 4 1]
Policy step 8: [0 0 5 2 5 0 5 0 1 5 0 0 3 1 1 1]
Policy step 9: [6 4 5 1 4 4 1 4 5 3 6 2 1 0 4 4]


## FSC Evaluation
If you extracted FSC, you can evaluate it in the simulator.

The action and update function have shapes [memory_size, nr_observations]. Action function provides played actions of Mealy machine for some memory and seen observation (integer index), update function tells us to which memory update to. 

In [56]:
memory_size = 3 # That means that the result is 3-FSC
action_function = tf.zeros((memory_size, stormpy_model.nr_observations), dtype=tf.int32) # Example of how to use the dummy action function
update_function = tf.zeros((memory_size, stormpy_model.nr_observations), dtype=tf.int32) # Example of how to use the dummy update function
action_labels = env.action_keywords # The action labels, that are used in the environment, i.e. if the first action is "up", the action 0 is "up"
  # Currently, this is not used in the proposed evaluate_policy_in_model function, but it is useful for PAYNT
  # TODO: Remapping action and update functions to the order of action labels

fsc_policy = TableBasedPolicy(
    original_policy=agent.policy(), # The policy of the agent, I use it because it have complete information about the environment specification
    external_observation_to_action_table=action_function,
    external_observation_to_update_table=update_function,
    action_keywords=action_labels,
    initial_memory = 0, # The initial memory of the policy, I use 0 because that is how it works in PAYNT, but you can set it to any value
)

evaluate_policy_in_model(
    fsc_policy,
    args,
    env,
    tf_env,
)
# NOTE: The evaluate_policy_in_model currently evaluates the policy only in 16 environments, that may not be enough for statistical significance.
#       If you want more environments, change the num_envs parameter in the ArgsEmulator class.
    


INFO:environment.environment_wrapper_vec:Resetting the environment.
INFO:tools.evaluation_results_class:Average Return = -80.776123046875
INFO:tools.evaluation_results_class:Average Virtual Goal Value = 31.34328269958496
INFO:tools.evaluation_results_class:Goal Reach Probability = 0.6268656716417911
INFO:tools.evaluation_results_class:Trap Reach Probability = 0.373134328358209
INFO:tools.evaluation_results_class:Variance of Return = 2518.6220703125
INFO:tools.evaluation_results_class:Current Best Return = -80.776123046875
INFO:tools.evaluation_results_class:Current Best Reach Probability = 0.6268656716417911


### Useful tools

One of the useful tools for FSC extraction is artificial synthesis of environment TimeSteps, since you can generate observations features valuation for all possible observations and then evaluate it on some policy. The observation integers are the same numbers as in StormPy observations.

In [57]:
fake_time_step = env.create_fake_timestep_from_observation_integer([4, 3, 1, 2])
print(fake_time_step) # Example of how to use the fake time step
initial_memory = fsc_policy.get_initial_state(4)
print()
print(fsc_policy.action(fake_time_step, initial_memory))

TimeStep(
{'step_type': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([1, 1, 1, 1], dtype=int32)>,
 'reward': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 'discount': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>,
 'observation': {'observation': <tf.Tensor: shape=(4, 8), dtype=float32, numpy=
array([[ 0.,  0.,  6.,  0., -1., -1.,  1.,  0.],
       [ 0.,  2.,  4.,  0.,  4.,  4.,  1.,  1.],
       [ 0.,  2.,  4.,  0., -1., -1.,  1.,  1.],
       [ 0.,  5.,  4.,  0., -1., -1.,  1.,  0.]], dtype=float32)>,
                 'mask': <tf.Tensor: shape=(4, 7), dtype=bool, numpy=
array([[ True, False, False, False, False, False, False],
       [False,  True, False,  True, False,  True,  True],
       [False,  True, False,  True, False,  True,  True],
       [ True, False, False, False, False, False, False]])>,
                 'integer': <tf.Tensor: shape=(4, 1), dtype=int32, numpy=
array([[4],
       [3],
     